In [3]:
pip install tpdm

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement tpdm (from versions: none)
ERROR: No matching distribution found for tpdm
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gensim

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 24.0 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 1.8 MB/s  eta 0:00:01
     |████████████████████████████████| 14.0 MB 12.0 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
# import fasttext
from gensim.models import FastText
import os
from tqdm import tqdm
import time
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)
import torch

In [3]:
!pip3 install spacy
!python3 -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 16.4 MB/s eta 0:00:0031m18.6 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
!pip3 install spacy

In [9]:
!python3 -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 44.3 MB/s eta 0:00:00 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [11]:
def preprocess_text(text):
    if not isinstance(text, str):
        if pd.isna(text):  
            return ""  
        try:
            text = str(text)  
        except:
            return ""  
    
    if not text.strip():
        return ""

    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return ' '.join(tokens)

In [12]:
import pandas as pd
pd.read_csv('data/jobs_embed.csv')

,jobTitle,companyName,finalZipcode,finalState,finalCity,companyBranchName,jobDescUrl,nlpBenefits,nlpSkills,nlpSoftSkills,nlpDegreeLevel,nlpEmployment,nlpSeniority,scrapedLocation,jobDesc,jobDesc_clean
0,"Nuclear Medicine Tech, Full Time, Day Shift",Adventist Health,93465,"CA,",Templeton,"Adventist Health || Templeton, CA, 93465",https://www.postjobfree.com/job/vubjwa/nuclear...,NaN,Radioactive Materials Radiology Information Sy...,Computer Literacy,Associates,Full-time,Entry level,"Templeton, CA, 93465",Job Description LOCATION SUMMARY: Located in T...,job description location summary located in te...
1,Radiation Therapist - SIGN-ON BONUS,SkinCure Oncology,47802,"IN,",Terre Haute,"SkinCure Oncology || Terre Haute, IN, 47802",https://www.postjobfree.com/job/vt9qzu/radiati...,Hsa/Fsa Vision Insurance Dental Insurance Eap ...,Hair Loss Radiation Protection Radiation Thera...,Customer Service,Masters Certificate,Full-time,Entry level,"Terre Haute, IN, 47802","Seeking Radiation Therapist: FULL-TIME, $5,000...",seeking radiation therapist fulltime 5000 sign...
2,Delivery Driver - Earn Extra Cash,DoorDash,46403,IN,Hobart,"DoorDash || Hobart, IN",https://us.jora.com/job/Delivery-Driver-36f004...,NaN,NaN,NaN,NaN,Full-time,Entry level,"Hobart, IN",Delivery Driver - Earn Extra Cash at DoorDash ...,delivery driver earn extra cash at doordash su...
3,Paramedic,AMR,11791,NY,Bay Shore,"AMR || Bay Shore, NY",https://us.jora.com/job/Paramedic-30a825beca41...,NaN,Medical Equipment Ambulances Agility,Management,High School,Full-time,Entry level,"Bay Shore, NY",More Information about this Job: Paramedic Bay...,more information about this job paramedic bay ...
4,Sales Lead - White House Black Market,Chico's,49315,"MI,",Byron Center,"Chico's || Byron Center, MI, 49315",https://www.postjobfree.com/job/vt7qu7/sales-l...,NaN,Loss Prevention Wardrobing Product Knowledge C...,Customer Service Operations Problem Solving Ma...,High School Associates,Full-time,Mid-Senior level,"Byron Center, MI, 49315","Join our fashion-obsessed team, dedicated to a...",join our fashionobsessed team dedicated to a s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97412,"Deputy Executive Director, External Affairs, M...",NYC Department of Health and Mental Hygiene,10129,NY,New York,NYC Department of Health and Mental Hygiene ||...,https://www.linkedin.com/jobs/view/deputy-exec...,NaN,Community Mental Health Public Health Strategi...,Communications,Bachelors,Full-time,Executive,"Manhattan, NY",The Mayor’s Office of Community Mental Health ...,the mayors office of community mental health o...
97413,Travel Cath Lab Technologist - per week,Springboard Healthcare,60461,"IL,",Olympia Fields,"Springboard Healthcare || Olympia Fields, IL, ...",https://www.postjobfree.com/job/vt9oc4/travel-...,NaN,Cath Lab Springboard,NaN,NaN,Contract,Mid-Senior level,"Olympia Fields, IL, 60461",Springboard Healthcare is seeking a travel Cat...,springboard healthcare is seeking a travel cat...
97414,Per Diem Clinical Quality Analyst - National R...,Optum,75168,TX,Waxahachie,"Optum || Waxahachie, TX",https://www.linkedin.com/jobs/view/per-diem-cl...,401K,Career Development Health Equity Auditing Dive...,Telecommuting Adaptability Critical Thinking P...,High School Certificate,Part-time,Mid-Senior level,"Waxahachie, TX",Optum is a global organization that delivers c...,optum is a global organization that delivers c...
97415,"Senior Superintendent, New Construction",Donohoe Companies,20853,MD,Bethesda,"Donohoe Companies || Bethesda, MD",https://us.jora.com/job/Senior-Superintendent-...,Paid Time Off 401K Remote Vision Insurance,Project Management Construction Activity Seque...,Operations Quality Control Management Forecast...,Masters,Temporary,Mid-Senior level,"Bethesda, MD",Overview Donohoe Construction Company is seeki...,overview donohoe construction company is seeki...


In [13]:
job_descriptions = pd.read_csv('data/jobs_embed.csv')['jobDesc_clean']

In [14]:
job_descriptions_spacy = job_descriptions.apply(preprocess_text)

In [16]:
df = pd.read_csv('data/jobs_embed.csv')

In [17]:
df['jobDesc_after_spacy'] = job_descriptions_spacy
df.to_csv('data/jobs_embed_clean.csv', index=False)


In [31]:
df = pd.read_csv('data/jobs_embed_clean.csv')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97417 entries, 0 to 97416
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   jobTitle             97417 non-null  object
 1   companyName          97417 non-null  object
 2   finalZipcode         97417 non-null  object
 3   finalState           97417 non-null  object
 4   finalCity            97417 non-null  object
 5   companyBranchName    97417 non-null  object
 6   jobDescUrl           97417 non-null  object
 7   nlpBenefits          60045 non-null  object
 8   nlpSkills            88200 non-null  object
 9   nlpSoftSkills        73358 non-null  object
 10  nlpDegreeLevel       57882 non-null  object
 11  nlpEmployment        97417 non-null  object
 12  nlpSeniority         97417 non-null  object
 13  scrapedLocation      97417 non-null  object
 14  jobDesc              97395 non-null  object
 15  jobDesc_clean        97395 non-null  object
 16  jobD

In [33]:
df.jobDesc_after_spacy.unique()

array(['job description location summary locate templeton oak tree adventist health twin city serve northern san luis obispo county comprise acute care facility team provide exceptional care emergency medicine orthopedic obstetric digestive disorder wound care medical surgical outpatient service local enjoy weekly farmer market downtown templeton farmtofork dining beautiful landscape hike biking beach day minute away cambria morro bay position summary role represent seiu responsible nuclear imaging newborn pediatric adolescent adult geriatric patient order physician post administration radioactive material position instruct patient nuclear medicine examination adjust imaging equipment determine proper dosage desire exposure time image follow alara guideline job requirement education work experience meet minimum requirement require licensure dhs require understand radiology information system utilization operation require computer literacy require license certification dhs certify techn

In [34]:
# create id column for this dataframe
df['id'] = df.index
df.to_csv('data/jobs_embed_clean.csv', index=False)
df = pd.read_csv('data/jobs_embed_clean.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97417 entries, 0 to 97416
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   jobTitle             97417 non-null  object
 1   companyName          97417 non-null  object
 2   finalZipcode         97417 non-null  object
 3   finalState           97417 non-null  object
 4   finalCity            97417 non-null  object
 5   companyBranchName    97417 non-null  object
 6   jobDescUrl           97417 non-null  object
 7   nlpBenefits          60045 non-null  object
 8   nlpSkills            88200 non-null  object
 9   nlpSoftSkills        73358 non-null  object
 10  nlpDegreeLevel       57882 non-null  object
 11  nlpEmployment        97417 non-null  object
 12  nlpSeniority         97417 non-null  object
 13  scrapedLocation      97417 non-null  object
 14  jobDesc              97395 non-null  object
 15  jobDesc_clean        97395 non-null  object
 16  jobD

In [35]:
df.jobDesc_after_spacy.unique()

array(['job description location summary locate templeton oak tree adventist health twin city serve northern san luis obispo county comprise acute care facility team provide exceptional care emergency medicine orthopedic obstetric digestive disorder wound care medical surgical outpatient service local enjoy weekly farmer market downtown templeton farmtofork dining beautiful landscape hike biking beach day minute away cambria morro bay position summary role represent seiu responsible nuclear imaging newborn pediatric adolescent adult geriatric patient order physician post administration radioactive material position instruct patient nuclear medicine examination adjust imaging equipment determine proper dosage desire exposure time image follow alara guideline job requirement education work experience meet minimum requirement require licensure dhs require understand radiology information system utilization operation require computer literacy require license certification dhs certify techn

In [36]:
#implement fasttext
from gensim.models import FastText

In [37]:
def train_fasttext(df):
    sentences = [text.split() if isinstance(text, str) else [] for text in df['jobDesc_after_spacy']]
    sentences = [s for s in sentences if s]
    model = FastText(sentences, vector_size=200, window=5, min_count=1, workers=4)
    return model

model = train_fasttext(df)
model.save('data/fasttext_model.bin')


In [38]:
#load fasttext model
model = FastText.load('data/fasttext_model.bin')

# generate the job description vectors
def generate_job_description_vectors(model, text):
    words = text.split() if isinstance(text, str) else []
    if not words:
        return np.zeros(model.vector_size)
    
    words_vectors = [model.wv[word] for word in words if word in model.wv]
    if not words_vectors:
        return np.zeros(model.vector_size)
    
    doc_vector = np.mean(words_vectors, axis=0)
    doc_vector = doc_vector / np.linalg.norm(doc_vector)
    return doc_vector.astype(np.float32)

job_descriptions_vectors = df['jobDesc_after_spacy'].apply(lambda x: generate_job_description_vectors(model, x))


In [39]:
job_descriptions_vectors = np.array(job_descriptions_vectors.tolist(), dtype=np.float32)
print(job_descriptions_vectors.shape) 

(97417, 200)


In [88]:
import os
from dotenv import load_dotenv
load_dotenv()

MILVUS_HOST = os.getenv("MILVUS_HOST")
MILVUS_PORT = os.getenv("MILVUS_PORT")
COLLECTION_NAME = os.getenv("COLLECTION_NAME")

connections.connect(alias="default", host=MILVUS_HOST, port=MILVUS_PORT)
print("Successfully connected to Milvus.")

vector_dim = job_descriptions_vectors.shape[1]

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="job_vector", dtype=DataType.FLOAT_VECTOR, dim=vector_dim)
]


collection = Collection(COLLECTION_NAME, schema=CollectionSchema(fields, "job description vectors"))
print(f"Collection {COLLECTION_NAME} created successfully.")



Successfully connected to Milvus.
Collection jobs_collection created successfully.


In [89]:
"""
collection = Collection(COLLECTION_NAME)
collection.drop()
print(f"Collection {COLLECTION_NAME} dropped successfully.")
"""


'\ncollection = Collection(COLLECTION_NAME)\ncollection.drop()\nprint(f"Collection {COLLECTION_NAME} dropped successfully.")\n'

In [91]:
# create index
index_params = {
    "index_type": "HNSW",
    "params": {
        "M": 16,
        "efConstruction": 64
    },
    "metric_type": "L2"
}
collection.create_index('job_vector',index_params)
collection.load()
print(f"Collection {COLLECTION_NAME} loaded successfully.")



Collection jobs_collection loaded successfully.


In [92]:
#insert data into milvus
batch_size = 5000 
total_vectors = len(job_descriptions_vectors)
job_ids = df['id'].tolist()

for i in range(0, total_vectors, batch_size):
    batch_vectors = job_descriptions_vectors[i:min(i+batch_size, total_vectors)]
    entities = [
        list(range(i, min(i+batch_size, total_vectors))),
        batch_vectors
    ]
    collection.insert(entities)
collection.flush()
print(f"Inserted {len(job_descriptions_vectors)} job description vectors into Milvus.")


Inserted 97417 job description vectors into Milvus.


In [93]:
# check the info of the collection
print(f"The number of entities in the collection is: {collection.num_entities}")

#load the collection
collection.load()
print(f"Collection {COLLECTION_NAME} loaded successfully.")

The number of entities in the collection is: 97417
Collection jobs_collection loaded successfully.


In [94]:
# search the collection
query_vector = job_descriptions_vectors[0]
top_k = 15

if isinstance(query_vector, np.ndarray):
    if len(query_vector.shape) == 1:
        query_vector = [query_vector.tolist()]
    else:
        query_vector = query_vector.tolist()
elif not isinstance(query_vector[0], list):
    query_vector = [query_vector]

search_params = {
    'metric_type': 'L2',
    'params': {'nprobe': 30}
}

results = collection.search(
    data=query_vector,
    anns_field="job_vector",
    param=search_params,
    limit=top_k,
    output_fields=["id"]
)

res = results[0]
for i, hit in enumerate(res):
    id = hit.entity.id
    distance = hit.distance+1
    print(f"Result {i+1}: ID={id}, Distance={distance}")


# save the results to a csv file
#results_df = pd.DataFrame(results)


Result 1: ID=0, Distance=1.0
Result 2: ID=9416, Distance=1.1520907878875732
Result 3: ID=1298, Distance=1.1536736488342285
Result 4: ID=2200, Distance=1.165724128484726
Result 5: ID=16548, Distance=1.180286467075348
Result 6: ID=8352, Distance=1.1804360747337341
Result 7: ID=990, Distance=1.1844389289617538
Result 8: ID=1456, Distance=1.1893317699432373
Result 9: ID=16261, Distance=1.1893440634012222
Result 10: ID=10305, Distance=1.1896536201238632
Result 11: ID=6697, Distance=1.1913165748119354
Result 12: ID=6669, Distance=1.1922723054885864
Result 13: ID=17200, Distance=1.1923773288726807
Result 14: ID=19719, Distance=1.1941371411085129
Result 15: ID=9803, Distance=1.1948100626468658


In [95]:

df['jobDesc_after_spacy'][:100]


0     job description location summary locate temple...
1     seek radiation therapist fulltime signon bonus...
2     delivery driver earn extra cash doordash summa...
3     information job paramedic bay shore ny start p...
4     join fashionobsesse team dedicate styling soci...
                            ...                        
95    customer service administrator responsible gen...
96    delivery driver doordash summary delivery driv...
97    pocatello set rocky mountain backdrop eastern ...
98    holden farm strive nation progressive innovati...
99    apply cover letter resume write sample require...
Name: jobDesc_after_spacy, Length: 100, dtype: object

In [99]:
precision_scores = []
recall_scores = []
ap_scores = []

def evaluate_search_system(test_queries, collection, top_k=15):
    
    model = FastText.load('data/fasttext_model.bin')
    for query, relevant_docs in test_queries.items():
        query_vector = generate_job_description_vectors(model, query)
        query_vector = [query_vector.tolist()] 
        search_params = {
        "metric_type": "L2",
        "params": {"nprobe": 30}
        }
        
        results = collection.search(
            data=query_vector,
            anns_field="job_vector",
            param=search_params,
            limit=top_k,
            output_fields=["id"]
        )
        
        if results is None:
            print(f"No results found for query: {query}")
        
        formatted_results = []
        for i, hits in enumerate(results):
            query_results = []
            for hit in hits:
                item = {
                    "id": hit.entity.id,
                    "distance": hit.distance
                }
                query_results.append(item)
            formatted_results.append(query_results)
            
        
        
        retrieved_docs = [hit['id'] for hit in formatted_results[0]]
        relevant_retrieved = set(retrieved_docs) & set(relevant_docs)
        
        
        
        precision = len(relevant_retrieved) / len(retrieved_docs) if retrieved_docs else 0
        precision_scores.append(precision)
        
        recall = len(relevant_retrieved) / len(relevant_docs) if relevant_docs else 0
        recall_scores.append(recall)
        
        ap = 0
        hits = 0
        
        for i, doc_id in enumerate(retrieved_docs):
            if doc_id in relevant_docs:
                hits += 1
                precision_at_k = hits / (i + 1) 
                ap += precision_at_k
        
        
        ap = ap / len(relevant_docs) if relevant_docs else 0
        ap_scores.append(ap)


    

In [100]:
import json
with open('data/search_test_dataset.json', 'r') as f:
    test_queries = json.load(f)
    
evaluate_search_system(test_queries, collection, top_k=5)
avg_precision = sum(precision_scores) / len(precision_scores) if precision_scores else 0
avg_recall = sum(recall_scores) / len(recall_scores) if recall_scores else 0
map_score = sum(ap_scores) / len(ap_scores) if ap_scores else 0
f1_score = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0

print(f'precision: {avg_precision}, recall: {avg_recall}, map: {map_score}, f1_score: {f1_score}, num_queries: {len(test_queries)}, queries_evaluated: {len(precision_scores)}')

precision: 0.35559495254978896, recall: 0.11112342267181145, map: 0.09541271248305144, f1_score: 0.1693309297856166, num_queries: 9589, queries_evaluated: 9589
